In [5]:
import pyodbc
import pandas as pd
from pathlib import Path
import glob
import datetime
import re
import os
import numpy as np
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from datetime import date
import math
import time

In [6]:
# home and time
home = Path.home()
todaystr = date.today().strftime('%Y-%m-%d')
# PNFV_alternative = pd.read_excel(Path(home, 'HP Inc', 'GPS TW Innovation - Documents', 'Users', 'GPS', 'PNFV', 'alternative.xlsx'))
# PNFV = pd.read_excel(Path(home, 'HP Inc', 'GPS TW Innovation - Documents', 'Users', 'GPS', 'PN FV description mapping table_ALL.xlsx'))
"""
currently the folder for buyer to update is still in GPSTW SOP - 2021 日新
REMEMBER to change the path when the folder officailly changed to GPS TW Innovation
"""
PNFV = pd.read_excel(Path(home, 'HP Inc', 'GPSTW SOP - 2021 日新', 'PN FV description mapping table_ALL.xlsx'))
# PNFV = pd.merge(PNFV, PNFV_alternative, on = 'Descr', how = 'left')

In [7]:
len(PNFV)

5539

In [8]:
start_time = time.time()
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()

cursor.execute(f"SELECT COUNT(*) FROM OPS.GPS_tbl_ops_PN_FV")
conn.commit()

cursor.execute(f"DELETE FROM OPS.GPS_tbl_ops_PN_FV")
conn.commit()
print("%s seconds ---" % (time.time() - start_time))

cursor.execute(f"SELECT COUNT(*) FROM OPS.GPS_tbl_ops_PN_FV")
conn.commit()

for index, row in PNFV.iterrows():

    Commodity = str(row['Commodity'])
    Supplier = str(row['Supplier'])
    PN = str(row['PN'])
    Descr = str(row['Descr'])
    #alternative = str(row['alternative part flag'])
    alternative = str(row['Alternative part flag'])

    cursor.execute(f"INSERT INTO CSI.OPS.GPS_tbl_ops_PN_FV ( Commodity, Supplier, PN, Descr, [alternative part flag] )\
                    VALUES('{Commodity}','{Supplier}','{PN}','{Descr}','{alternative}')")
    
    print("%s seconds ---" % (time.time() - start_time))
conn.commit()
conn.close()
print("%s seconds ---" % (time.time() - start_time))

2.8740410804748535 seconds ---
3.439711809158325 seconds ---
3.6235997676849365 seconds ---
3.8064889907836914 seconds ---
3.985900640487671 seconds ---
4.166246175765991 seconds ---
4.348677396774292 seconds ---
4.530950307846069 seconds ---
4.711991786956787 seconds ---
4.893622875213623 seconds ---
5.0744452476501465 seconds ---
5.273430585861206 seconds ---
5.452641010284424 seconds ---
5.632641315460205 seconds ---
5.8138439655303955 seconds ---
5.99257493019104 seconds ---
6.173895359039307 seconds ---
6.353010177612305 seconds ---
6.535062789916992 seconds ---
6.716456174850464 seconds ---
6.896260976791382 seconds ---
7.0782694816589355 seconds ---
7.257444620132446 seconds ---
7.438160181045532 seconds ---
7.638584613800049 seconds ---
7.817864894866943 seconds ---
8.000326871871948 seconds ---
8.180658340454102 seconds ---
8.361305236816406 seconds ---
8.545390367507935 seconds ---
8.727088451385498 seconds ---
8.908233880996704 seconds ---
9.090050220489502 seconds ---
9.271